<a href="https://colab.research.google.com/github/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Simplificação de Texto

Nas primeiras abordagens, houve uma tentativa de produzir uma rede neural que tivesse como entrada o *token frequence* e saida outro *token frequence*. Apesar disso poder ser considerado um resultado válido para a tentativa de melhorar a classificação de texto, não podemos considerar uma frase como um conjunto de palavras com a propriedade da comutatividade. A ordem das palavras acaba importando muito para os humanos.

## importando bugingangas

In [0]:
import numpy as np
import csv
import pandas as pd
from urllib import request as req
from gensim.models import keyedvectors as kv

## Paper With Code

~~Após um gole de sorte, eu acabo por encontrar essa maravilha~~ chamado de [Paper With Code](https://paperswithcode.com/sota/document-summarization-on-cnn-daily-mail), é um ~~bom~~ compilado de resultados cientificos sobre determinados problemas da computação. O primeiro artigo do link de cima, mostra uma solução com ROGUE-1 de 43.83 para o problema de sumarização de documentos como o *state of the art*.

![paper with code](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/paperwithcode.png?raw=true)
<center>paper with code</center>

No artigo *Text Summarization with Pretrained Encoders* (LIU e LAPATA, 2019), os autores representam o texto como *Bert*, que é gerado por uma rede neural e é um caso especial do *word2vec*.




## Representação do Bert word2vec

*Kyubyong* criou um preset do Bert, um dicionário onde cada palavra é representada por um vetor de 768 valores decimais. Visto algumas limitações das plataformas *Google Colab* e do *Github*, serão usadas apenas 300 instâncias do CNN, que gerou um dicionário reduzido de apenas 25mb.

![Bert is Evil](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/audio-banner.jpg?raw=true)
<center>Bert is Evil</center>

## Base de dados CNN

A base de dados pode ser encontrado [nesse github da google](https://github.com/google-research-datasets/sentence-compression), no formato *Json*. Essa base de dados é formada com notícias da CNN com a primeira linha da noticia, com o título, com todos os bigramas possíveis e com informações do NER. Porém no trabalho apenas será usado a primeira linha como entrada e o título como saída.

![Json](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/Jasonf.jpg?raw=true)
<center>Json</center>

## Abrindo uma noticia (bert) e convertendo para texto.

In [34]:
# Abrindo o dicionario
dicurl = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/dicionario.csv'
#dicloader = req.urlopen(dicurl)
dicio = []

with req.urlopen(dicurl) as f:
  #print(f.read()[0:20000].decode())
  meucsv = f.read().decode('charmap')
  meucsv = meucsv.split('\n')
  print(meucsv[0])
  #data = csv.reader(meucsv)
  print(type(data))
  print(len(meucsv))
  for i in meucsv:
    dicio.append(i)

print(dicio[0])

#abrindo a noticia
#nn = input("digite um numero entre 0 e 199: ")



five,-0.00221736589,-0.000327187416,-0.024606755,-0.0176957119,0.0173947439,-0.0174167212,-0.0306010358,-0.00145815569,0.0187633559,-0.0108142281,0.0432017148,-0.0502034388,-0.00846801512,-0.0576570071,0.0322442949,-0.020834649,-0.0111211119,0.00136801903,0.00872333068,0.0229032859,-0.0216196161,-0.0165795088,0.0135502145,-0.0349892341,-0.0627112761,0.0271395817,0.0384537391,-0.00406109542,-0.0425047055,-0.0520481579,-0.00754025206,-0.0537302122,0.00043773267,-0.0310549233,-0.0576889291,-0.0168994479,0.0107453763,0.012590304,-0.0184273627,-0.0116608078,-0.0242777597,-0.0558727495,-0.0306073502,-0.0593236089,-0.0174500663,0.00768886693,-0.0218660533,0.0144082364,0.00758720376,0.0240487028,0.00155591127,0.026569115,-0.00286716945,0.0214601867,0.00812067464,-0.0130317695,0.0433732457,-0.061860837,-0.0274253748,0.0190580823,-0.0728581473,-0.0425258093,-0.0313723385,-0.0307020862,-0.0634120479,0.00250072358,-0.0189387277,-0.0331520401,-0.0607296601,0.0383252986,0.0210423823,-0.0433877259,-0

## Referencias

